In [1]:
import numpy as np
from tribarmes import TriangleBarMesh

In [2]:
fname = "stlsamples/frameguide.stl"
#fname = "stlsamples/onshapetestpart.stl"
tbm = TriangleBarMesh(fname)
vs = tbm.GetBarMeshTriangles()

# quickly make these as independent triangles
vv = np.array(vs).ravel()
X, Y, Z = vv[0::3], vv[1::3], vv[2::3]
tris = np.array(range(len(vs)*3))
tris.resize((len(vs), 3))


In [3]:
# define a cutter shape
cornerrad, flatrad, shaftheight = 5, 1, 38
px, py, pz = 20, -20, 0

tr = np.linspace(0, cornerrad+1, cornerrad*10+1)
theta = np.linspace(0, 2*np.pi, 40)
tr, theta = np.meshgrid(tr, theta)
def toolheight(tr):
    return cornerrad - np.sqrt(np.maximum(0, cornerrad**2 - tr**2)) + \
                       np.maximum(0, (tr-cornerrad)*(shaftheight))
tX = np.minimum(tr+flatrad, cornerrad+flatrad) * np.sin(theta)
tY = np.minimum(tr+flatrad, cornerrad+flatrad) * np.cos(theta)
tZ = toolheight(tr)

from matplotlib.tri import Triangulation
ttri = Triangulation(np.ravel(tr), np.ravel(theta))
ttris = ttri.triangles
    

In [4]:
import ipyvolume as ipv

def rendersurftool(px, py):
    k = ipv.view()
    print(k)
    ipv.figure()
    ipv.xyzlim(-40,40)
    mesh = ipv.plot_trisurf(X, Y, Z, triangles=tris, color='orange')
    ipv.plot_trisurf(tX.ravel()+px, tY.ravel()+py, tZ.ravel()+pz, triangles=ttris, color="red")
    ipv.view(*k)
    ipv.show()

rendersurftool(20, -20)
#ipv.save("/home/julian/repositories/transition-CAM/thing.html", offline=True)


(0.0, 0.0, 2.0)


In [5]:
from ipywidgets import FloatSlider, interact, HBox, interactive_output

fsx = FloatSlider(value=20, min=-50, max=50, step=1, description="xpos", continuous_update=False)
fsy = FloatSlider(value=-20, min=-50, max=50, step=1, description="ypos", continuous_update=False)

out = interactive_output(rendersurftool, {"px":fsx, "py":fsy})
display(HBox([fsx, fsy]), out)


Output()

In [6]:
import ipyvolume as ipv
import numpy as np
x, y, z, u, v, w = np.random.random((6, 1000))*2-1
selected = np.random.randint(0, 1000, 100)
ipv.figure()

quiver = ipv.quiver(x, y, z, u, v, w, size=5, size_selected=8, selected=selected)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0, max=30, step=0.1)
size_selected = FloatSlider(min=0, max=30, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((quiver, 'size'), (size, 'value'))
jslink((quiver, 'size_selected'), (size_selected, 'value'))
jslink((quiver, 'color'), (color, 'value'))
jslink((quiver, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])

In [7]:
ipv.figure()
quiver = ipv.scatter(x, y, z, u, v, w, size=5, size_selected=8, selected=selected)


TypeError: scatter() got multiple values for argument 'size'

In [8]:
help(ipv.scatter)

Help on function scatter in module ipyvolume.pylab:

scatter(x, y, z, color='red', size=2, size_selected=2.6, color_selected='white', marker='diamond', selection=None, grow_limits=True, **kwargs)
    Plots many markers/symbols in 3d
    
    :param x: numpy array of shape (N,) or (S, N) with x positions. If an (S, N) array, the first dimension will be used for frames in an animation.
    :param y: idem for y
    :param z: idem for z
    :param color: color for each point/vertex/symbol, can be string format, examples for red:'red', '#f00', '#ff0000' or 'rgb(1,0,0), or rgb array of shape (N, 3 or 4) or (S, N, 3 or 4)
    :param size: float representing the size of the glyph in percentage of the viewport, where 100 is the full size of the viewport
    :param size_selected: like size, but for selected glyphs
    :param color_selected:  like color, but for selected glyphs
    :param marker: name of the marker, options are: 'arrow', 'box', 'diamond', 'sphere', 'point_2d', 'square_2d', 'trian

In [9]:
ipv.figure()
ipv.scatter(x[:10], y[:10], z[:10], size=50, marker="cat")
ipv.show()


In [10]:
help(ipv.scatter)

Help on function scatter in module ipyvolume.pylab:

scatter(x, y, z, color='red', size=2, size_selected=2.6, color_selected='white', marker='diamond', selection=None, grow_limits=True, **kwargs)
    Plots many markers/symbols in 3d
    
    :param x: numpy array of shape (N,) or (S, N) with x positions. If an (S, N) array, the first dimension will be used for frames in an animation.
    :param y: idem for y
    :param z: idem for z
    :param color: color for each point/vertex/symbol, can be string format, examples for red:'red', '#f00', '#ff0000' or 'rgb(1,0,0), or rgb array of shape (N, 3 or 4) or (S, N, 3 or 4)
    :param size: float representing the size of the glyph in percentage of the viewport, where 100 is the full size of the viewport
    :param size_selected: like size, but for selected glyphs
    :param color_selected:  like color, but for selected glyphs
    :param marker: name of the marker, options are: 'arrow', 'box', 'diamond', 'sphere', 'point_2d', 'square_2d', 'trian

In [11]:
exit

In [11]:

import ipyvolume.datasets
stream = ipyvolume.datasets.animated_stream.fetch()
print("shape of steam data", stream.data.shape) # first dimension contains x, y, z, vx, vy, vz


shape of steam data (6, 200, 1250)


In [14]:
import ipyvolume as ipv
fig = ipv.figure()
# instead of doing x=stream.data[0], y=stream.data[1], ... vz=stream.data[5], use *stream.data
# limit to 50 timesteps to avoid having a huge notebook
q = ipv.quiver(*stream.data[:,0:50,:200], color="red", size=7, marker="arrow")
ipv.style.use("dark") # looks better
ipv.animation_control(q, interval=200)
q.geo = "box"
ipv.show()

In [13]:
q.geo

'arrow'

In [31]:
import ipyvolume as ipv
ipv._doc_snippets

AttributeError: module 'ipyvolume' has no attribute '_doc_snippets'

In [36]:
help(ipv.pylab.scatter)

Help on function scatter in module ipyvolume.pylab:

scatter(x, y, z, color='red', size=2, size_selected=2.6, color_selected='white', marker='diamond', selection=None, grow_limits=True, **kwargs)
    Plots many markers/symbols in 3d
    
    :param x: numpy array of shape (N,) or (S, N) with x positions. If an (S, N) array, the first dimension will be used for frames in an animation.
    :param y: idem for y
    :param z: idem for z
    :param color: color for each point/vertex/symbol, can be string format, examples for red:'red', '#f00', '#ff0000' or 'rgb(1,0,0), or rgb array of shape (N, 3 or 4) or (S, N, 3 or 4)
    :param size: float representing the size of the glyph in percentage of the viewport, where 100 is the full size of the viewport
    :param size_selected: like size, but for selected glyphs
    :param color_selected:  like color, but for selected glyphs
    :param marker: name of the marker, options are: 'arrow', 'box', 'diamond', 'sphere', 'point_2d', 'square_2d', 'trian

In [20]:
import random
x = [random.random()  for i in range(1000000) ]

In [21]:
import numpy as np
y = np.array(x)
y

array([0.86884161, 0.60995721, 0.22621818, ..., 0.55810634, 0.21812434,
       0.62288882])

In [28]:
y[:3] + [90, 92, 93]

array([  90.86884161,   92.60995721, 9092.        ])

In [33]:
y.shape = (10,100,1000)

In [36]:
y[1,1,1]

0.3407239723675167

In [39]:
dir(y)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_e